# Курсовой проект для курса "Python для Data Science"
#### Исполнитель Васильев А.

### Задание:
Используя данные из обучающего датасета (train.csv), построить модель для предсказания цен на недвижимость (квартиры).  
С помощью полученной модели, предсказать цены для квартир из тестового датасета (test.csv).

In [ ]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score as r2
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings('ignore')

In [ ]:
TRAIN_DATASET_PATH = 'datasets/train.csv'
TEST_DATASET_PATH = 'datasets/test.csv'
PREDICT_PRICE_PATH  = 'datasets/AVsasilev_predictions.csv'

In [ ]:
title_font = {
    "fontsize": 16,
    "fontweight": "bold",
    "color": "darkgrey",
    "family": "arial",
}

label_font = {
    "fontsize": 10,
    "family": "arial",
}

In [ ]:
def r2_score(train_true, train_pred, test_true, test_pred):
    print('Train R2:\t' + str(round(r2(train_true, train_pred), 3)) + '\n' +
         'Test R2:\t' + str(round(r2(test_true, test_pred), 3)) + '\n')
    
    plt.figure(figsize=(10,5))
    
    plt.subplot(121)
    plt.scatter(x=train_true, y=train_pred)
    plt.title('Train true vs Predicted values', fontdict=title_font)
    plt.xlabel('Train predicted values', fontdict=label_font)
    plt.ylabel('Train true values', fontdict=label_font)
    
    plt.subplot(122)
    plt.scatter(x=test_true, y=test_pred)
    plt.title('Test true vs Predicted values', fontdict=title_font)
    plt.xlabel('Test predicted values', fontdict=label_font)
    plt.ylabel('Test true values', fontdict=label_font)
        
    plt.show()

### Описание датасета
1. Id - идентификационный номер квартиры
2. DistrictId - идентификационный номер района
3. Rooms - количество комнат
4. Square - площадь
5. LifeSquare - жилая площадь
6. KitchenSquare - площадь кухни
7. Floor - этаж
8. HouseFloor - количество этажей в доме
9. HouseYear - год постройки дома
10. Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
11. Social_1, Social_2, Social_3 - социальные показатели местности
12. Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
13. Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
14. Price - цена квартиры

### Загружаю датасеты

#### Train dataset

In [ ]:
df_train = pd.read_csv(TRAIN_DATASET_PATH)
print(f'Форма обучающего датасета:\t{df_train.shape}')

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
print(f'Количество пропущенных значений:\n{df_train.isnull().sum()}')

#### Test dataset

In [ ]:
df_test = pd.read_csv(TEST_DATASET_PATH)
print(f'Форма тестового датасета:\t{df_test.shape}')

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
print(f'Количество пропущенных значений:\n{df_test.isnull().sum()}')

### Обработка пропусков

#### LifeSquare

In [ ]:
lsquare_median_train = round(df_train["LifeSquare"].median(), 6)
lsquare_median_test = round(df_test["LifeSquare"].median(), 6)
print(f'Медиана LifeSquare обучающего датасета:\t {lsquare_median_train}')
print(f'Медиана LifeSquare тестового датасета:\t {lsquare_median_test}')

Пропуски заполняю медианным значением LifeSquare, если это значение меньше значения Square

In [ ]:
df_train.loc[df_train['LifeSquare'].isnull() & (df_train['Square'] > lsquare_median_train), 'LifeSquare'] = lsquare_median_train

In [ ]:
df_test.loc[df_test['LifeSquare'].isnull() & (df_train['Square'] > lsquare_median_test), 'LifeSquare'] = lsquare_median_test

Оставшиеся пропуски заполняю разницей Square и разницы медианных значений Square и LifeSquare, если это значение меньше значения LifeSquare

In [ ]:
sub_square_train = round(df_train['Square'].median() - df_train['LifeSquare'].median(), 6)
print(f'Разница медиан Square и LifeSquare обучающего датасета:\t {sub_square_train}')

In [ ]:
df_train.loc[df_train['LifeSquare'].isnull() & (df_train['Square'] > sub_square_train), 'LifeSquare'] = df_train['Square'] - lsquare_median_train

In [ ]:
sub_square_test = round(df_test['Square'].median() - df_test['LifeSquare'].median(), 6)
print(f'Разница медиан Square и LifeSquare тестового датасета:\t {sub_square_test}')

In [ ]:
df_test.loc[df_test['LifeSquare'].isnull() & (df_test['Square'] > sub_square_test), 'LifeSquare'] = df_test['Square'] - lsquare_median_test

Последнее пустое значение обучающего датасета приравниваю к Square

In [ ]:
df_train.loc[df_train['LifeSquare'].isnull()]

In [ ]:
df_train.loc[df_train['LifeSquare'].isnull(), 'LifeSquare'] = df_train['Square']

In [ ]:
print(f'Количество пустых значений признака LifeSquare обучающего датасета:\t{df_train.loc[df_train["LifeSquare"].isnull(), "LifeSquare"].sum()}')
print(f'Количество пустых значений признака LifeSquare тестового датасета:\t{df_test.loc[df_test["LifeSquare"].isnull(), "LifeSquare"].sum()}')

##### Healthcare_1

In [ ]:
hc1_median_train = round(df_train["Healthcare_1"].median(), 3)
hc1_median_test = round(df_test["Healthcare_1"].median(), 3)
print(f'Медиана Healthcare_1 обучающего датасета:\t {hc1_median_train}')
print(f'Медиана Healthcare_1 тестового датасета:\t {hc1_median_test}')

Пустые значения заполняю медианами

In [ ]:
df_train.loc[df_train['Healthcare_1'].isnull(), 'Healthcare_1'] = hc1_median_train

In [ ]:
df_test.loc[df_test['Healthcare_1'].isnull(), 'Healthcare_1'] = hc1_median_test

In [ ]:
print(f'Количество пустых значений признака Healthcare_1 обучающего датасета:\t{df_train.loc[df_train["Healthcare_1"].isnull(), "Healthcare_1"].sum()}')
print(f'Количество пустых значений признака Healthcare_1 тестового датасета:\t{df_test.loc[df_test["Healthcare_1"].isnull(), "Healthcare_1"].sum()}')

### Scatterplot для визуализации разброса стоимости квартир от общей площади

In [ ]:
plt.figure(figsize=(10,6))

sns.scatterplot(x=df_train['Price'], y=df_train['Square'])

plt.title('Square & Price', fontdict=title_font)
plt.ylabel('Square', fontdict=label_font)
plt.xlabel('Price', fontdict=label_font)

plt.show()

### Обработка выбросов обучающего датасета

In [ ]:
df_train.describe()

#### Rooms
Значениям больше 10 и равные 0 присваиваю значение медианы

In [ ]:
df_train.loc[(df_train['Rooms'] > 10) | (df_train['Rooms'] == 0)]

In [ ]:
df_train.loc[(df_train['Rooms'] > 10) | (df_train['Rooms'] == 0), 'Rooms'] = df_train['Rooms'].median()

#### Square
Значениям больше 200 или меньше 15 присваиваю значение произведения количества комнат квартиры и средней общей площади квартиры с 1 комнатой

In [ ]:
averange_square_per_room = round(df_train['Square'].mean() / df_train['Rooms'].mean(), 6)
print(f'Средняя общая площадь квартиры с 1 комнатой:\t{averange_square_per_room}')

In [ ]:
df_train.loc[(df_train['Square'] > 200) | (df_train['Square'] < 15), 'Square'] = df_train['Rooms'] * averange_square_per_room

#### LifeSquare
Значениям больше Square и меньше либо равным 10 присваиваю значение разности Square и sub_square_train  
Получившиеся отрицательные или равные нулю значения заполняю значением медианы

In [ ]:
# Данные значения заполняю разностью Square и sub_square_train
df_train.loc[(df_train['LifeSquare'] > df_train['Square']) | (df_train['LifeSquare'] <= 10), 'LifeSquare'] = df_train['Square'] - sub_square_train

In [ ]:
# Отрицательным значениям присваиваю среднее значения для квартир с общей площадью до 20 м2
df_train.loc[df_train['LifeSquare'] <= 0, 'LifeSquare'] = df_train.loc[df_train['LifeSquare'] <= 20, 'LifeSquare'].mean()

#### KitchenSquare
Значениям более 150 или менее 5 присваиваю значение медианы

In [ ]:
df_train.loc[(df_train['KitchenSquare'] > 150) | (df_train['KitchenSquare'] <= 5), 'KitchenSquare'] = df_train['KitchenSquare'].median()

#### HouseYear

In [ ]:
df_train.loc[df_train['HouseYear'] > 2020]

In [ ]:
df_train.loc[df_train['HouseYear'] == 20052011.0, 'HouseYear'] = 2005

In [ ]:
df_train.loc[df_train['HouseYear'] == 4968.0, 'HouseYear'] = 1968

#### HouseFloor
Значения меньше этажа, на которой находится квартира, заменяю на разность медиан HouseFloor и Floor

In [ ]:
df_train.loc[df_train['HouseFloor'] < df_train['Floor'], 'HouseFloor'] = df_train['HouseFloor'].median() - df_train['Floor'].median()

In [ ]:
df_train.describe()

### Обработка выбросов тестового датасета

In [ ]:
df_test.describe()

#### Rooms
Значения более 10 и равные 0 заполняю значением медианы

In [ ]:
df_test.loc[(df_test['Rooms'] > 10) | (df_test['Rooms'] == 0)]

In [ ]:
df_test.loc[(df_test['Rooms'] > 10) | (df_test['Rooms'] == 0), 'Rooms'] = df_test['Rooms'].median()

#### Square
Значениям больше 200 или меньше 15 присваиваю значение произведения количества комнат квартиры и средней общей площади квартиры с 1 комнатой

In [ ]:
averange_square_per_room = round(df_test['Square'].mean() / df_test['Rooms'].mean(), 6)
print(f'Средняя общая площадь квартиры с 1 комнатой:\t{averange_square_per_room}')

In [ ]:
df_test.loc[(df_test['Square'] > 200) | (df_test['Square'] < 15), 'Square'] = df_test['Rooms'] * averange_square_per_room

#### LifeSquare
Значениям больше Square и меньше либо равным 10 присваиваю значение разности Square и sub_square_train  
Получившиеся отрицательные или равные нулю значения заполняю средним значением для квартир с общей площадью до 20 м2 

In [ ]:
df_test.loc[(df_test['LifeSquare'] > df_test['Square']) | (df_test['LifeSquare'] <= 10), 'LifeSquare'] = df_test['Square'] - sub_square_test

In [ ]:
df_test.loc[df_test['LifeSquare'] <= 0, 'LifeSquare'] = df_test.loc[df_test['LifeSquare'] <= 20, 'LifeSquare'].mean()

#### KitchenSquare
Значениям более 10 или менее 4 присваиваю значение медианы

In [ ]:
df_test.loc[(df_test['KitchenSquare'] > 130) | (df_test['KitchenSquare'] <= 4), 'KitchenSquare'] = df_test['KitchenSquare'].median()

#### HouseFloor
Значения меньше этажа, на которой находится квартира, заменяю на разность медиан HouseFloor и Floor

In [ ]:
df_test.loc[df_test['HouseFloor'] < df_test['Floor'], 'HouseFloor'] = df_test['HouseFloor'].median() - df_test['Floor'].median()

In [ ]:
df_test.describe()

### Обрабатываю object-признаки

#### Обучающий датасет

In [ ]:
df_train.dtypes

In [ ]:
df_train['Ecology_2'].value_counts()

In [ ]:
df_train['Ecology_3'].value_counts()

In [ ]:
df_train['Shops_2'].value_counts()

Заменяю категориальные признаки бинарными данными

In [ ]:
df_train['Ecology_2_bin'] = df_train['Ecology_2'].replace({'A':0, 'B':1})
df_train['Ecology_3_bin'] = df_train['Ecology_3'].replace({'A':0, 'B':1})
df_train['Shops_2_bin'] = df_train['Shops_2'].replace({'A':0, 'B':1})

Изменяю тип признака `Id` на `str`

In [ ]:
df_train['Id'] = df_train['Id'].astype(str)

In [ ]:
df_train.head()

In [ ]:
df_train.dtypes

#### Тестовый датасет

Заменяю категориальные признаки бинарными данными

In [ ]:
df_test['Ecology_2_bin'] = df_test['Ecology_2'].replace({'A':0, 'B':1}).astype(int)
df_test['Ecology_3_bin'] = df_test['Ecology_3'].replace({'A':0, 'B':1}).astype(int)
df_test['Shops_2_bin'] = df_test['Shops_2'].replace({'A':0, 'B':1}).astype(int)

Изменяю тип признака `Id` на `str`

In [ ]:
df_train['Id'] = df_train['Id'].astype(str)

In [ ]:
df_test.head()

### Добавляю дополнительные признаки

#### Признаки:  
- DistrictSize - размер района  
- IsDistrictLarge - бинарный признак, указывающий является ли данный район большим или нет

##### Обучающий датасет

In [ ]:
df_train['DistrictId'].value_counts()

In [ ]:
district_size_train = df_train['DistrictId'].value_counts().reset_index()\
               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size_train.head()

In [ ]:
df_train = df_train.merge(district_size_train, on='DistrictId', how='left')
df_train.head()

In [ ]:
(df_train['DistrictSize'] > 100).value_counts()

In [ ]:
df_train['IsDistrictLarge'] = (df_train['DistrictSize'] > 100).astype(int)

In [ ]:
df_train.head()

##### Тестовый датасет

In [ ]:
df_test['DistrictId'].value_counts()

In [ ]:
district_size_test = df_test['DistrictId'].value_counts().reset_index()\
               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size_test.head()

In [ ]:
df_test = df_test.merge(district_size_test, on='DistrictId', how='left')
df_test.head()

In [ ]:
(df_test['DistrictSize'] > 100).value_counts()

In [ ]:
df_test['IsDistrictLarge'] = (df_test['DistrictSize'] > 100).astype(int)

In [ ]:
df_test.head()

#### MedPriceByDistrict  - медиана стоимости квартир по районам

#### Обучающий датасет

In [ ]:
med_price_by_district = df_train.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                       .rename(columns={'Price':'MedPriceByDistrict'})

med_price_by_district.head()

In [ ]:
df_train = df_train.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
df_train.head()

##### Переношу признак на тестовый датасет

In [ ]:
df_test = df_test.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
df_test.info()

Пустые значения заполняю медианами MedPriceByDistrict районов аналогичного размера

In [ ]:
for i in df_test.index[df_test['MedPriceByDistrict'].isnull()]:
    district_size = df_test.iloc[i, 22]
    df_test.iat[i, 24] = df_test.loc[(df_test['DistrictSize'] == district_size) & (~df_test['MedPriceByDistrict'].isnull()), 'MedPriceByDistrict'].median()

In [ ]:
df_test.info()

### Таблица корреляции признаков обучающего датасета

In [ ]:
plt.figure(figsize = (13,10))

sns.set(font_scale=0.5)
sns.heatmap(df_train.corr(), annot=True, linewidths=.5, cmap='GnBu')

plt.title('Correlation matrix', fontdict=title_font)

plt.show()

### Стандартизирую признаки

#### Обучающий датасет

In [ ]:
df_train.dtypes

In [ ]:
columns_for_stand = df_train.columns[:19]

In [ ]:
columns_for_stand = df_train[columns_for_stand].select_dtypes(include=['float64', 'int64']).columns.tolist()
columns_for_stand.append('DistrictSize')
columns_for_stand.append('MedPriceByDistrict')
columns_for_stand

In [ ]:
scaler = StandardScaler()
standard_features = scaler.fit_transform(df_train[columns_for_stand])

In [ ]:
df_train_scaled = df_train.copy()
df_train_scaled[columns_for_stand] = pd.DataFrame(standard_features, columns=columns_for_stand)

In [ ]:
df_train_scaled.head()

In [ ]:
df_train_scaled.std()

#### Тестовый датасет

In [ ]:
standard_features_test = scaler.transform(df_test[columns_for_stand])

In [ ]:
df_test_scaled = df_test.copy()
df_test_scaled[columns_for_stand] = pd.DataFrame(standard_features_test, columns=columns_for_stand)

In [ ]:
df_test_scaled.head()

In [ ]:
df_test_scaled.std()

### Удаляю object-признаки

#### Обучающий датасет

In [ ]:
feature_names = df_train_scaled[df_train.columns].select_dtypes(include=['float64', 'int64', 'uint8']).columns.tolist()
feature_names

In [ ]:
df_train_prepared = df_train_scaled[feature_names]
df_train_prepared.head()

#### Тестовый датасет

In [ ]:
feature_names.remove('Price')
feature_names

In [ ]:
df_test_prepared = df_test_scaled[feature_names]
df_test_prepared.head()

### Разбиваю обучающий датасет на X и y

In [ ]:
X = df_train_prepared[feature_names]
y = df_train_prepared['Price']

### Разбиваю обучающий датасет на кластеры для получения дополнительного признака

In [ ]:
def Scatter_3D(components, labels=None):
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(components[:, 0], components[:, 1], 
               components[:, 2], c=labels, cmap=plt.get_cmap('jet'), alpha=0.5)

    plt.title('3D mapping of objects')
    plt.show()

In [ ]:
tsne_3D = TSNE(n_components=3, learning_rate=250, random_state=27)
X_tsne_3D = tsne_3D.fit_transform(X)
print(f'Shape of X_train_tsne_3D dataframe: {X_tsne_3D.shape}')

In [ ]:
kmeans = KMeans(n_clusters=5, max_iter=100, random_state=27)
X_kmeans_3D = kmeans.fit_predict(X)

In [ ]:
Scatter_3D(X_tsne_3D, X_kmeans_3D)

In [ ]:
X['Cluster'] = X_kmeans_3D

In [ ]:
X.head()

### Разбиваю обучающий датасет на train и valid

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=27)

In [ ]:
X_train.head()

### Обучаю модель GradientBoostingRegressor

Использую GradientBoostingRegressor т.к. она показала наилучший результат по сравнению с другими моделями регресии

In [ ]:
final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=27)
final_model.fit(X_train, y_train)

In [ ]:
y_train_preds = final_model.predict(X_train)
y_valid_preds = final_model.predict(X_valid)
r2_score(y_train, y_train_preds, y_valid, y_valid_preds)

### Важность признаков

In [ ]:
feature_importances = pd.DataFrame(zip(X_train.columns, final_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

### Делаю предсказание на тестовом датасете

In [ ]:
test_preds = final_model.predict(df_test_prepared)

##### Поле `Id` и предсказанное поле `Price` представляю в виде датасета

In [ ]:
predict_price = pd.DataFrame(df_test['Id'])

In [ ]:
predict_price['Price'] = test_preds

In [ ]:
print(f'Форма массива с предсказанной ценой:\t{predict_price.shape}')

In [ ]:
predict_price.head(10)

#### Гистограмма с ценами квартир из обучающего датасета в сравнении с предсказанными моделью

In [ ]:
plt.figure(figsize=(6,4))
df_train['Price'].hist(alpha=0.5)
predict_price['Price'].hist(alpha=0.5)

plt.title('Train and predicted prices', fontdict=title_font)
plt.ylabel('count', fontdict=label_font)
plt.xlabel('Price', fontdict=label_font)

plt.show()

### Перекрёстная проверка

In [ ]:
cv_score = cross_val_score(final_model, X, y, scoring='r2', cv=KFold(n_splits=5, shuffle=True, random_state=27))
cv_score

In [ ]:
print('Среднее значение: ', cv_score.mean())
print('Стандартное отклонение: ', cv_score.std())
print('Среднее + отклонение: ', cv_score.mean() - cv_score.std())
print('Среднее - отклонение: ', cv_score.mean() + cv_score.std())

### Сохраняю получившийся датасет в файл .csv

In [ ]:
predict_price.to_csv(PREDICT_PRICE_PATH, index=False, encoding='utf-8')